In [ ]:
import logging
import asyncio
import pandas as pd
from IPython.display import display, Markdown

from datetime import datetime

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors import TelegrafHTTPEventProcessor, RedisEventProcessor
from tastytrade.config import RedisConfigManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

# Service Connections

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

# Market Data Subscriptions

In [ ]:
start_time = datetime(2025, 3, 19)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX", "/ESH25:XCME"]
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

In [ ]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

# Check | Market Data feeds

In [ ]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
display(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames[f"{symbol}"]
    .tail(5)
    .sort(by="time", descending=True)
)

In [ ]:
display(Markdown(f"**Quote Feed:** {symbol}"))

display(dxlink.router.handler[Channels.Quote].processors["feed"].pl)

In [6]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()